permits 
what does the housing look like
1:5 rule, what does a new block look like
consider a local example, or focusing on 1 county
section 8 vouchers

In [2]:
import pandas as pd
import glob

# set display options to show all rows
pd.set_option('display.max_rows', None)

# Define the path to the directory containing the Excel files
path = r'source files'

# Get a list of file names in the directory that end with .xlsx
file_list = glob.glob(path + "/*.xlsx")

# Create an empty list to hold the DataFrames
df_list = []

# Loop over the file names and read each file into a DataFrame
for file in file_list:
    df = pd.read_excel(file, header=1)
    df = df.dropna(axis=1, how='all')
    df = df[~df['o'].isin([0.1,0.2, 0.3])]


    df['units'] = pd.to_numeric(df['units'], errors='coerce')
    df_list.append(df)

# Concatenate the DataFrames into a single DataFrame
combined_df = pd.concat(df_list)

# Group the data by the "comu" column and sum the "unit" values
sum_by_comu = combined_df.groupby('comu')['units'].sum()

# Create a new DataFrame with the summed values
result_df = pd.DataFrame({'comu': sum_by_comu.index.astype(int), 'total_unit': sum_by_comu.values})


result_df.head(50)

,comu,total_unit
0,101,403.0
1,102,4813.0
2,103,13.0
3,104,238.0
4,107,327.0
5,108,472.0
6,111,549.0
7,112,213.0
8,113,176.0
9,114,5.0


In [3]:
import pandas as pd
import glob

# NOTE SOME OF THE MUNI CODES MIGHT BE OFF BY 1 IN ORIGINAL SHEETS
# NOTE right now the "total_units" of municipalities with no entry are set to "0". I don't actually know if they had zero units, or it is a hole in datasets. 

# set display options to show all rows
pd.set_option('display.max_rows', None)

# Define the path to the directory containing the Excel files
path = r'source files'

# Get a list of file names in the directory that end with .xlsx
file_list = glob.glob(path + "/*.xlsx")

# Create an empty list to hold the DataFrames
df_list = []

# Loop over the file names and read each file into a DataFrame
for file in file_list:
    df = pd.read_excel(file, header=1)
    df = df.dropna(axis=1, how='all')
    df = df[~df['o'].isin([0.1,0.2, 0.3])]


    df['units'] = pd.to_numeric(df['units'], errors='coerce')
    df_list.append(df)

# Concatenate the DataFrames into a single DataFrame
combined_df = pd.concat(df_list)

# Group the data by the "comu" column and sum the "unit" values
sum_by_comu = combined_df.groupby('comu')['units'].sum()

# Create a new DataFrame with the summed values
result_df = pd.DataFrame({'comu': sum_by_comu.index, 'total_unit': sum_by_comu.values})

# Read in the other dataset with the "Muni Code" column
other_df = pd.read_excel("jacobson files\simplified data 7.xlsx")

# Merge the two datasets on the common columns "comu" and "Muni Code"
merged_df = pd.merge(result_df, other_df, how='right', left_on='comu', right_on='Muni Code')

# Fill in missing "total_unit" values with 0
merged_df['total_unit'] = merged_df['total_unit'].fillna(0)

# Convert the "total_unit" column to integers
merged_df['total_unit'] = merged_df['total_unit'].astype(int)

# Drop the "comu" column. can omit this code to see the column matching
merged_df = merged_df.drop(columns=['comu'])

# Read in the "settled towns and prosp need.xlsx" dataset
settled_towns_df = pd.read_excel("jacobson files\settled towns and prosp need.xlsx", usecols=["Municode", "#s including VLA and Durational Adjustment"])
settled_towns_df = settled_towns_df.rename(columns={"Municode": "Muni Code"})
# settled_towns_df['Muni Code'] = settled_towns_df['Muni Code'].astype(int)

# Merge the "settled_towns_df" with the "merged_df" using a left join
merged_df = pd.merge(merged_df, settled_towns_df, how='left', on='Muni Code')

# Fill in missing values in the "#s including VLA and Durational Adjustment" column with "no value"
merged_df['#s including VLA and Durational Adjustment'] = merged_df['#s including VLA and Durational Adjustment'].fillna('unknown value')

merged_df.head(20)

,total_unit,Muni Code,Municipality,County,Region,Prior Round Obligations (1987-1999),Present Need (2015),Gap Present Need (2015),Prospective Need (2015-2025),Total,Prospective + Gap Present,#s including VLA and Durational Adjustment
0,403,101,Absecon City,Atlantic,6,144,51,40,15,250,55,94.0
1,4813,102,Atlantic City City,Atlantic,6,2458,828,0,0,3286,0,unknown value
2,13,103,Brigantine City,Atlantic,6,124,38,28,175,365,203,0.0
3,238,104,Buena Borough,Atlantic,6,41,7,10,15,73,25,43.0
4,0,105,Buena Vista Township,Atlantic,6,19,67,22,11,119,33,0.0
5,0,106,Corbin City,Atlantic,6,13,1,8,14,36,22,0.0
6,327,107,Egg Harbor City,Atlantic,6,42,52,16,0,110,16,unknown value
7,472,108,Egg Harbor Township,Atlantic,6,763,101,289,251,1404,540,1000.0
8,0,109,Estell Manor City,Atlantic,6,21,0,15,10,46,25,unknown value
9,0,110,Folsom Borough,Atlantic,6,20,1,15,10,46,25,unknown value


In [4]:
total_units_sum = merged_df['total_unit'].sum()

print(total_units_sum)

206076


In [5]:
# set display options to show all rows
pd.set_option('display.max_rows', None)

# Define the path to the directory containing the Excel files
path = r'source files'

# Get a list of file names in the directory that end with .xlsx
file_list = glob.glob(path + "/*.xlsx")

# Create an empty list to hold the DataFrames
df_list = []
final_data_list = []  # add a new empty list

# Loop over the file names and read each file into a DataFrame
for file in file_list:
    df = pd.read_excel(file, header=1)
    df = df.dropna(axis=1, how='all')
    df = df[~df['o'].isin([0.1,0.2, 0.3])]
   
   # this all generages the "units column"
    df['units'] = pd.to_numeric(df['units'], errors='coerce')
    df_list.append(df)

    # this generates the "comu" and "source" columns
    # select the columns "comu" and "final data" from the DataFrame
    combined_data = df[["comu", "source"]]
    # append the selected columns to the list
    final_data_list.append(combined_data)


# combine/concatinate the "units" rows from every spreadsheet
combined_df = pd.concat(df_list)
# combine/concatinate the "comu", and "source" rows from every spreadsheet
final_data_df = pd.concat(final_data_list)


# Group the data by the "comu" column and sum the "unit" values
sum_by_comu = combined_df.groupby('comu')['units'].sum()

# Create a new DataFrame with the summed values
result_df = pd.DataFrame({'comu': sum_by_comu.index, 'total_unit': sum_by_comu.values})

# Merge the final data DataFrame with the result_df DataFrame on the "comu" column using a left join
result_df = pd.merge(result_df, final_data_df, how='left', on='comu')

# Read in the other dataset with the "Muni Code" column
other_df = pd.read_excel("jacobson files\simplified data 7.xlsx")

# Merge the two datasets on the common columns "comu" and "Muni Code"
merged_df = pd.merge(result_df, other_df, how='right', left_on='comu', right_on='Muni Code')

# Fill in missing "total_unit" values with 0
merged_df['total_unit'] = merged_df['total_unit'].fillna(0)

# Convert the "total_unit" column to integers
merged_df['total_unit'] = merged_df['total_unit'].astype(int)

# Drop the "comu" column. can omit this code to see the column matching
merged_df = merged_df.drop(columns=['comu'])

# Read in the "settled towns and prosp need.xlsx" dataset
settled_towns_df = pd.read_excel("jacobson files\settled towns and prosp need.xlsx", usecols=["Municode", "#s including VLA and Durational Adjustment"])
settled_towns_df = settled_towns_df.rename(columns={"Municode": "Muni Code"})
# settled_towns_df['Muni Code'] = settled_towns_df['Muni Code'].astype(int)

# Merge the "settled_towns_df" with the "merged_df" using a left join
merged_df = pd.merge(merged_df, settled_towns_df, how='left', on='Muni Code')

# Fill in missing values in the "#s including VLA and Durational Adjustment" column with "no value"
merged_df['#s including VLA and Durational Adjustment'] = merged_df['#s including VLA and Durational Adjustment'].fillna('unknown value')

merged_df.head(100)

,total_unit,source,Muni Code,Municipality,County,Region,Prior Round Obligations (1987-1999),Present Need (2015),Gap Present Need (2015),Prospective Need (2015-2025),Total,Prospective + Gap Present,#s including VLA and Durational Adjustment
0,403,HUD,101,Absecon City,Atlantic,6,144,51,40,15,250,55,94.0
1,403,HUD,101,Absecon City,Atlantic,6,144,51,40,15,250,55,94.0
2,403,HUD,101,Absecon City,Atlantic,6,144,51,40,15,250,55,94.0
3,403,tax credit,101,Absecon City,Atlantic,6,144,51,40,15,250,55,94.0
4,403,HUD,101,Absecon City,Atlantic,6,144,51,40,15,250,55,94.0
5,4813,PHA,102,Atlantic City City,Atlantic,6,2458,828,0,0,3286,0,unknown value
6,4813,MtL,102,Atlantic City City,Atlantic,6,2458,828,0,0,3286,0,unknown value
7,4813,tax credit,102,Atlantic City City,Atlantic,6,2458,828,0,0,3286,0,unknown value
8,4813,PHA,102,Atlantic City City,Atlantic,6,2458,828,0,0,3286,0,unknown value
9,4813,HUD,102,Atlantic City City,Atlantic,6,2458,828,0,0,3286,0,unknown value
